In [103]:
from bs4 import BeautifulSoup
from multiprocessing import Process
import requests
import threading
import pandas as pd
import time
import concurrent.futures
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', -1)
import re

In [2]:
url = 'https://kontakt.az/'
content = requests.get(url)
soup = BeautifulSoup(content.text,'html.parser')

In [3]:
category_links = []
paginated_links = []
products_links = []


def get_category_links(soup):
    for i in soup.find_all('a',class_='menuliparent_a'):
        if i.get('href') not in category_links:
                category_links.append(i.get('href'))
    return category_links


def pagination(category_links):
    for link in category_links:
        content_2 = requests.get(link)
        soup_2 = BeautifulSoup(content_2.text,'html.parser')
        try:
            b = soup_2.find('span',class_='pages').text.split(' / ')[1]
            a = soup_2.find('span',class_='pages').text.split(' / ')[0]
        except:
            b = 1
            a = 1
        for i in range(int(a),int(b)+1):
            paginated_links.append(link + 'page/' + str(i) + '/')
    return paginated_links


def get_prod_links(paginated_links):
    for urls in paginated_links[1:9]:
        content_3 = requests.get(urls)
        soup_3 = BeautifulSoup(content_3.text,'html.parser')
        a = soup_3.find_all('a',draggable='false')
        for url in a:
            products_links.append(url.get('href'))
    return products_links

In [4]:
get_category_links(soup)
category_links.remove('https://kontakt.az/telefonlar/mobil-telefonlar/xiaomi-mobil-telefonlar/')
category_links.remove('https://kontakt.az/telefonlar/mobil-telefonlar/apple-mobil-telefonlar/')
category_links.remove('https://kontakt.az/telefonlar/mobil-telefonlar/samsung-mobil-telefonlar/')
pagination(category_links)
get_prod_links(paginated_links)

['https://kontakt.az/vivo-y16-3-32-gb-stellar-black/',
 'https://kontakt.az/vivo-v25-8-256-gb-aquamarine-blue/',
 'https://kontakt.az/vivo-v25-8-256-gb-diamond-black/',
 'https://kontakt.az/vivo-v25-pro-12-256-gb-surfing-blue/',
 'https://kontakt.az/vivo-v25-pro-12-256-gb-starlight-black/',
 'https://kontakt.az/vivo-v25e-8-128-gb-sunrise-gold/',
 'https://kontakt.az/vivo-v25e-8-128-gb-diamond-black/',
 'https://kontakt.az/vivo-y35-4-64-gb-dawn-gold/',
 'https://kontakt.az/vivo-y35-4-64-gb-agate-black/',
 'https://kontakt.az/vivo-y22-4-64-gb-starlit-blue/',
 'https://kontakt.az/huawei-nova-10-pro-8-256-gb-starry-silver/',
 'https://kontakt.az/xiaomi-poco-m5-6-128-gb-green/',
 'https://kontakt.az/huawei-nova-10-se-8-128-gb-mint-green/',
 'https://kontakt.az/huawei-nova-10-se-8-128-gb-starry-black/',
 'https://kontakt.az/huawei-nova-10-8-128gb-starry-black/',
 'https://kontakt.az/huawei-nova-10-8-128gb-starry-silver/',
 'https://kontakt.az/tecno-camon-19-6-128-gb-seasalt-white/',
 'https:

In [117]:
df = pd.DataFrame()
prop = []
link = []
code = []
name = []
price = []
cash_price = []
category = []
sub_category = []
for i in products_links[1:30]:
    e = requests.get(i)
    s = BeautifulSoup(e.text,'html.parser')
    link.append(i)
    code.append(s.find('div',class_='product-status').find('span').text)
    name.append(s.find('h1',class_='title').text)
    price.append(s.find('span',class_='nprice').text)
    category.append(s.find('div',class_='col-12').find_all('a')[1].text)
    sub_category.append(s.find('div',class_='col-12').find_all('a')[2].text)
    try:
        pr = re.findall('\d+',s.find('div',class_='discounted-price').text)
        cash_price.append(float(pr[0])+float(pr[1])/100)
    except:
        pr = None
        cash_price.append(pr)
    pc = s.find('div',class_='product-status').find('span').text
    c = s.find_all('ul',class_='features-review-tabs submenuItems_params')[0].text
    v = s.find_all('ul',class_='features-review-tabs submenuItems_params')[1].text
    c = c.replace('\n\n','').replace('\n ','').replace('\n',',').split(',')
    v = v.replace('\n\n','').replace('\n ','').replace('\n',',').split(',')
    sd = dict(zip(c,v))
    prop.append(sd)
df = pd.DataFrame(category,columns=['Category'])
df['Sub_Category'] = sub_category
df['Code'] = code
df['Link'] = link
df['Name'] = name
df['Price'] = price
df['Discounted_Price'] = cash_price
df['Properties'] = prop
df['Sub_Category'] = sub_category

In [118]:
df

,Category,Sub_Category,Code,Link,Name,Price,Discounted_Price,Properties
0,Telefonlar və planşetlər,Smartfonlar,TM-DG-SBP-1105-SM-2117,https://kontakt.az/vivo-v25-8-256-gb-aquamarin...,Vivo V25 8/256 GB Aquamarine Blue,1149.99,NaN,"{'Ekran': '6.44""', 'Daxili yaddaş': '8 GB', 'O..."
1,Smartfonlar,Vivo,TM-DG-SBP-1105-SM-2116,https://kontakt.az/vivo-v25-8-256-gb-diamond-b...,Vivo V25 8/256 GB Diamond Black,1149.99,NaN,"{'Ekran': '6.44""', 'Daxili yaddaş': '256 GB', ..."
2,Telefonlar və planşetlər,Smartfonlar,TM-DG-SBP-1105-SM-2115,https://kontakt.az/vivo-v25-pro-12-256-gb-surf...,Vivo V25 Pro 12/256 GB Surfing Blue,1299.99,NaN,"{'Ekran': '6.56""', 'Daxili yaddaş': '256 GB', ..."
3,Smartfonlar,Vivo,TM-DG-SBP-1105-SM-2114,https://kontakt.az/vivo-v25-pro-12-256-gb-star...,Vivo V25 Pro 12/256 GB Starlight Black,1299.99,NaN,"{'Ekran': '6.56""', 'Daxili yaddaş': '256 GB', ..."
4,Smartfonlar,Vivo,TM-DG-SBP-1105-SM-2113,https://kontakt.az/vivo-v25e-8-128-gb-sunrise-...,Vivo V25E 8/128 GB Sunrise Gold,799.99,NaN,"{'Ekran': '6.44""', 'Daxili yaddaş': '128 GB', ..."
5,Smartfonlar,Vivo,TM-DG-SBP-1105-SM-2112,https://kontakt.az/vivo-v25e-8-128-gb-diamond-...,Vivo V25e 8/128 GB Diamond Black,799.99,NaN,"{'Ekran': '6.44""', 'Daxili yaddaş': '128 GB', ..."
6,Smartfonlar,Vivo,TM-DG-SBP-1105-SM-2109,https://kontakt.az/vivo-y35-4-64-gb-dawn-gold/,Vivo Y35 4/64 GB Dawn Gold,449.99,NaN,"{'Ekran': '6.58""', 'Daxili yaddaş': '64 GB', '..."
7,Smartfonlar,Vivo,TM-DG-SBP-1105-SM-2108,https://kontakt.az/vivo-y35-4-64-gb-agate-black/,Vivo Y35 4/64 GB Agate Black,449.99,NaN,"{'Ekran': '6.58""', 'Daxili yaddaş': '64 GB', '..."
8,Smartfonlar,Vivo,TM-DG-SBP-1105-SM-2106,https://kontakt.az/vivo-y22-4-64-gb-starlit-blue/,Vivo Y22 4/64 GB Starlit Blue,399.99,NaN,"{'Ekran': '6.55""', 'Daxili yaddaş': '64 GB', '..."
9,Smartfonlar,Huawei,TM-DG-SBP-1105-SM-2092,https://kontakt.az/huawei-nova-10-pro-8-256-gb...,HUAWEI Nova 10 Pro 8/256 GB Starry Silver,1599.99,1399.99,"{'Ekran': '6.78""', 'Daxili yaddaş': '256 GB', ..."
